In [1]:
import pandas as pd
import numpy as np
import matplotlib as mp
import seaborn as sb

# Importing the required packages
from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 

In [2]:
hangul_syllables = np.array([chr(code) for code in range(44032, 55204)])
hangul_syllables = hangul_syllables.reshape(19, 21, 28)

print(f"'가'와 관련된 음절 리스트: {hangul_syllables[0][0]}")
print(f"'개'와 관련된 음절 리스트: {hangul_syllables[0][1]}")

print()

print(f"'ㄱ'와 관련된 마지막 음절 리스트: {hangul_syllables[0][20]}")
print(f"'ㄲ'와 관련된 첫번째 음절 리스트: {hangul_syllables[1][0]}")

'가'와 관련된 음절 리스트: ['가' '각' '갂' '갃' '간' '갅' '갆' '갇' '갈' '갉' '갊' '갋' '갌' '갍' '갎' '갏' '감' '갑'
 '값' '갓' '갔' '강' '갖' '갗' '갘' '같' '갚' '갛']
'개'와 관련된 음절 리스트: ['개' '객' '갞' '갟' '갠' '갡' '갢' '갣' '갤' '갥' '갦' '갧' '갨' '갩' '갪' '갫' '갬' '갭'
 '갮' '갯' '갰' '갱' '갲' '갳' '갴' '갵' '갶' '갷']

'ㄱ'와 관련된 마지막 음절 리스트: ['기' '긱' '긲' '긳' '긴' '긵' '긶' '긷' '길' '긹' '긺' '긻' '긼' '긽' '긾' '긿' '김' '깁'
 '깂' '깃' '깄' '깅' '깆' '깇' '깈' '깉' '깊' '깋']
'ㄲ'와 관련된 첫번째 음절 리스트: ['까' '깍' '깎' '깏' '깐' '깑' '깒' '깓' '깔' '깕' '깖' '깗' '깘' '깙' '깚' '깛' '깜' '깝'
 '깞' '깟' '깠' '깡' '깢' '깣' '깤' '깥' '깦' '깧']


In [3]:
'''
chosung_list = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ',
                'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
'''
chosung_list_test = ['ㄱ', 'ㄴ', 'ㄷ',  'ㄹ', 'ㅁ', 'ㅂ',  'ㅅ',
                'ㅇ', 'ㅈ',  'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']


jungsung_list = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ',
                 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']

jongsung_list = ['', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ',
                 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ',
                 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

In [4]:
def get_unicode_number(chosung_index, jungsung_index, jongsung_index):
    return ((chosung_index * 588) + (jungsung_index * 28) + jongsung_index) + 44032


# 박 / ㅂ:7 / ㅏ:0 / ㄱ:1
print(get_unicode_number(7, 0, 1), chr(get_unicode_number(7, 0, 1)))

# 놔 / ㄴ:2 / ㅘ:9 / '':0
print(get_unicode_number(2, 9, 0), chr(get_unicode_number(2, 9, 0)))

# 밝 / ㅂ:7 / ㅏ:0 / ㄺ:9
print(get_unicode_number(7, 0, 9), chr(get_unicode_number(7, 0, 9)))

48149 박
45460 놔
48157 밝


In [5]:
import re

class SeparateJaMo:
    """
    한글 자모를 분리하는 클래스

    target_text = 자모를 분리할 문자열
    blank_str = 공백을 처리할 문자
    remove_blank = 공백 제거 여부
    remove_special_character = 특수 문자 제거 여부
    refine_blank = 두개 이상의 공백 정제 여부
    refine_english = 영어 정제 여부
    """

    def __init__(self, original_text, blank_str=" ",
                 remove_blank=False, remove_special_character=False,
                 refine_blank=False, refine_english=False):
        self.__chosung_list = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ',
                               'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
        self.__jungsung_list = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ',
                                'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
        self.__jongsung_list = ['', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ',
                                'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ',
                                'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
        self.__original_text = original_text
        self.__blank_str = blank_str
        self.__remove_blank = remove_blank
        self.__remove_special_character = remove_special_character
        self.__refine_blank = refine_blank
        self.__refine_english = refine_english
        self.__processed_text = self.__make_processed_text()
        self.__jamo_list = self.__make_jamo_list()

    def __make_processed_text(self):
        """
        옵션에 알맞게 문자열을 정제하여 반환
        """
        processed_text = self.__original_text
        if self.__refine_blank:
            processed_text = " ".join(
                [word.strip() for word in processed_text.split(" ") if not len(word.strip()) == 0]
            )
        if self.__remove_blank:
            processed_text = re.sub(r'\s', '', processed_text)
        if self.__remove_special_character:
            processed_text = re.sub(r'[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', processed_text)
            processed_text = re.sub(r'\W \S', '', processed_text)
        if self.__refine_english:
            processed_text = re.sub(r'[^ㄱ-ㅎㅣ가-힣]+', '', processed_text)
        processed_text = re.sub(r'\s', self.__blank_str, processed_text)
        return processed_text

    def __make_jamo_list(self):
        """
        전체 자모 리스트 반환
        """
        jamo_list = list()
        for syllable in list(self.__processed_text):
            if re.match(r'[ㄱ-ㅎㅣ가-힣]+', syllable):
                syllable_code = ord(syllable)
                chosung_index = int((syllable_code - 44032) / 588)
                jungsung_index = int((syllable_code - 44032 - (chosung_index * 588)) / 28)
                jongsung_index = int(syllable_code - 44032 - (chosung_index * 588) - (jungsung_index * 28))
                jamo_list.append(
                    [
                        self.__chosung_list[chosung_index],
                        self.__jungsung_list[jungsung_index],
                        self.__jongsung_list[jongsung_index]
                    ]
                )
            else:
                jamo_list.append([syllable, syllable, syllable])
        return jamo_list

    def get_full_jamo_list(self):
        return self.__jamo_list

    def get_chosung_list(self):
        return [jamo_list[0] for jamo_list in self.__jamo_list]

    def get_jungsung_list(self):
        return [jamo_list[1] for jamo_list in self.__jamo_list]

    def get_jongsung_list(self):
        return [jamo_list[2] for jamo_list in self.__jamo_list]


if __name__ == "__main__":
    text = 'facebook의 창시자 마크 저커버그는       "사람과 사람의 연결이 곧 비즈니스" 라고 말했다.'
    separate_jamo = SeparateJaMo(text, remove_special_character=True)
    print(separate_jamo.get_full_jamo_list())
    print(separate_jamo.get_chosung_list())
    print(separate_jamo.get_jungsung_list())
    print(separate_jamo.get_jongsung_list())

[['f', 'f', 'f'], ['a', 'a', 'a'], ['c', 'c', 'c'], ['e', 'e', 'e'], ['b', 'b', 'b'], ['o', 'o', 'o'], ['o', 'o', 'o'], ['k', 'k', 'k'], ['ㅇ', 'ㅢ', ''], [' ', ' ', ' '], ['ㅊ', 'ㅏ', 'ㅇ'], ['ㅅ', 'ㅣ', ''], ['ㅈ', 'ㅏ', ''], [' ', ' ', ' '], ['ㅁ', 'ㅏ', ''], ['ㅋ', 'ㅡ', ''], [' ', ' ', ' '], ['ㅈ', 'ㅓ', ''], ['ㅋ', 'ㅓ', ''], ['ㅂ', 'ㅓ', ''], ['ㄱ', 'ㅡ', ''], ['ㄴ', 'ㅡ', 'ㄴ'], [' ', ' ', ' '], [' ', ' ', ' '], [' ', ' ', ' '], [' ', ' ', ' '], [' ', ' ', ' '], ['ㄹ', 'ㅏ', 'ㅁ'], ['ㄱ', 'ㅘ', ''], [' ', ' ', ' '], ['ㅅ', 'ㅏ', ''], ['ㄹ', 'ㅏ', 'ㅁ'], ['ㅇ', 'ㅢ', ''], [' ', ' ', ' '], ['ㅇ', 'ㅕ', 'ㄴ'], ['ㄱ', 'ㅕ', 'ㄹ'], ['ㅇ', 'ㅣ', ''], [' ', ' ', ' '], ['ㄱ', 'ㅗ', 'ㄷ'], [' ', ' ', ' '], ['ㅂ', 'ㅣ', ''], ['ㅈ', 'ㅡ', ''], ['ㄴ', 'ㅣ', ''], ['ㅅ', 'ㅡ', ''], [' ', ' ', ' '], ['ㄹ', 'ㅏ', ''], ['ㄱ', 'ㅗ', ''], [' ', ' ', ' '], ['ㅁ', 'ㅏ', 'ㄹ'], ['ㅎ', 'ㅐ', 'ㅆ'], ['ㄷ', 'ㅏ', '']]
['f', 'a', 'c', 'e', 'b', 'o', 'o', 'k', 'ㅇ', ' ', 'ㅊ', 'ㅅ', 'ㅈ', ' ', 'ㅁ', 'ㅋ', ' ', 'ㅈ', 'ㅋ', 'ㅂ', 'ㄱ', 'ㄴ', ' ', ' ', ' ', ' ', ' ', 'ㄹ', 'ㄱ', ' ', '